In [78]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#	Select Important Features and create a dataframe 

In [79]:
#droped the 'ever_married' and 'Residence_type' column and loaded the csv on dataFrame
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
data=data[['gender',	'age',	'hypertension'	,'heart_disease',	'work_type'	,'avg_glucose_level','bmi','smoking_status','stroke']]
data.head()

,gender,age,hypertension,heart_disease,work_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Private,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Self-employed,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Private,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Private,171.23,34.4,smokes,1
4,Female,79.0,1,0,Self-employed,174.12,24.0,never smoked,1


In [80]:
#fiels the NaN fields of bmi with Median of entire bmi column
data['bmi']=data['bmi'].fillna(data['bmi'].median())


In [81]:
#Removed outliers from age and bmi columns
cols=['age','bmi']
q1=data[cols].quantile(0.25)
q2 = data[cols].quantile(0.75)
a = q2-q1
data = data[~((data[cols] < (q1 - 1.5 * a)) |(data[cols] > (q2 + 1.5 * a))).any(axis=1)]


In [82]:
#MinMaxScalar preprocessing done on the columns to bring the values of the columns in certain range
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
data[['avg_glucose_level','bmi']]=scalar.fit_transform(data[['avg_glucose_level','bmi']])

In [83]:
#LabelEncoder applied on genders  bexause it contains only 3 distint values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['gender'] = le.fit_transform(data['gender'])


In [84]:
import numpy as np
#applying onehot encoding becaues both the columns had more than 2 distinct values
x=pd.get_dummies(data,columns=['smoking_status','work_type'])
data=pd.concat([data,x],axis=1)
data=data.drop(['smoking_status','work_type'],axis=1)
data = data.loc[:,~data.columns.duplicated()].copy()


In [85]:
l=data.pop('stroke')
data['stroke']=l


In [86]:
data['bmi']=data['bmi'].fillna(data['bmi'].median())
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values


##	Split Data to train, test, validate 

In [87]:
#Spliting the dataset in test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)


In [88]:
#Declared the features for tunning og the random forest algo

n_estimatiors=[int(x) for x in np.linspace(start=10,stop=200,num=20)]
max_features=['sqrt']
max_depth=[2,4]
min_sample_split=[2,5]
min_sample_leaf=[1,2]
bootstrap=[True,False]

param_grid={
    'n_estimators':n_estimatiors,
    'max_features':max_features,
    'max_depth':max_depth,
    'min_samples_split':min_sample_split,
    'min_samples_leaf':min_sample_leaf,
    'bootstrap':bootstrap
    
}


## Train Model 

In [89]:
#Trained the dataset on Random Forest and used GridSearchCV for aTunning
#IT tested the algorithm on 3*320=960 different parameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
classifier = RandomForestClassifier()
classifier=GridSearchCV(estimator=classifier,param_grid=param_grid,cv=3,verbose=2,n_jobs=4 )
classifier.fit(X_train, y_train)

Fitting 3 folds for each of 320 candidates, totalling 960 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4],
                         'max_features': ['sqrt'], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100, 110, 120, 130, 140, 150, 160,
                                          170, 180, 190, 200]},
             verbose=2)

In [90]:
classifier.best_estimator_

RandomForestClassifier(max_depth=2, n_estimators=10)

In [91]:
print (f'Train Accuracy - : {classifier.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {classifier.score(X_test,y_test):.3f}')

Train Accuracy - : 0.950
Test Accuracy - : 0.953


In [92]:
y_pred = classifier.predict(X_test)
print(y_pred)

[0 0 0 ... 0 0 0]


In [93]:
#Finded the accuracy on the test cases
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1187    0]
 [  59    0]]


0.9526484751203852

In [94]:
#Created the pickle file
import pickle
with open('model_pkl', 'wb') as files:
    pickle.dump(classifier, files)

In [95]:
with open('model_pkl' , 'rb') as f:
    lr = pickle.load(f)
y_val_predict=lr.predict(X_test)


In [96]:
#created the pandas dataframe for the csv creation
rd=pd.DataFrame(X_test,columns=['gender', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level',
       'bmi', 'smoking_status_Unknown',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes', 'work_type_0', 'work_type_1', 'work_type_2',
       'work_type_3', 'work_type_4'])
rd['Stroke']=y_test
rd['stroke_predicted']=y_pred

In [97]:
rd.to_csv('validation-predicted.csv', index=False)